In [1]:
import dlib
import cv2
import numpy as np
import face_recognition
import os
from pathlib import Path
import pandas as pd
from PIL import Image

In [2]:
#Function to obtain the encodings from face recognition library

def get_encoding(path):
    x=np.array(Image.open(path))
    x=cv2.cvtColor(x,cv2.COLOR_BGR2RGB)
    #face=face_recognition.face_locations(x)[0]
    try:
        enc=face_recognition.face_encodings(x)[0]
    except:
        enc=x
        print(path)
    #print(enc)
    
    return enc

In [3]:

rootdir = Path('C:/c_program/face/img')
l=[]
df = pd.DataFrame(columns = ['x', 'label'])
  

#apeending the image paths to a list    
for root, subdirectories, files in os.walk(rootdir):
    for file in files:
        x=os.path.join(root, file)
        l.append(x)
        
        


In [4]:
#Obtaining the features and adding them to a dataframe
for i in l:
        p=i.split("\\")
        
        enc=get_encoding(i)
        #print(p[4])
        df=df.append({'x' : enc, 'label' : p[4]},  
                ignore_index = True)

In [5]:
#Transforming labels into numbers
print(df)
from sklearn import preprocessing 
df['label'].unique() 

# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'label'. 
df['label']= label_encoder.fit_transform(df['label']) 
  
df['label'].unique()

                                                     x                  label
0    [-0.03787478804588318, 0.2088412493467331, 0.1...  Arnold_Schwarzenegger
1    [-0.06490906327962875, 0.19593630731105804, 0....  Arnold_Schwarzenegger
2    [-0.033288076519966125, 0.23150882124900818, 0...  Arnold_Schwarzenegger
3    [-0.009613871574401855, 0.1634233295917511, 0....  Arnold_Schwarzenegger
4    [-0.06668805330991745, 0.16968196630477905, 0....  Arnold_Schwarzenegger
..                                                 ...                    ...
143  [-0.009106200188398361, 0.16770702600479126, 0...         Vladimir_Putin
144  [-0.009661173447966576, 0.18971280753612518, 0...         Vladimir_Putin
145  [-0.017986586317420006, 0.08232659101486206, 0...         Vladimir_Putin
146  [0.04461508244276047, 0.08460748195648193, 0.0...         Vladimir_Putin
147  [-0.0066591366194188595, 0.12526565790176392, ...         Vladimir_Putin

[148 rows x 2 columns]


array([0, 1, 2, 3, 4])

In [6]:
#Saving the dataset 
df.to_csv('file1.csv', index=False) 

In [7]:
#Obtaining arrays to pass through ANN
dataset = pd.read_csv('file1.csv')
x=[]
y=[]
#print(dataset)
for ind in df.index: 
    l=list(df['x'][ind])
    x.append(l)
    y.append(df['label'][ind])
x=np.array(x)
y=np.array(y)
#print(x)

                                                     x  label
0    [-0.03787479  0.20884125  0.11381891 -0.073479...      0
1    [-6.49090633e-02  1.95936307e-01  9.58812609e-...      0
2    [-3.32880765e-02  2.31508821e-01  1.84001014e-...      0
3    [-0.00961387  0.16342333  0.09833182 -0.039257...      0
4    [-0.06668805  0.16968197  0.14714023 -0.037930...      0
..                                                 ...    ...
143  [-0.0091062   0.16770703  0.04390327 -0.045627...      4
144  [-0.00966117  0.18971281  0.04138543 -0.060494...      4
145  [-0.01798659  0.08232659  0.02026807 -0.058085...      4
146  [ 0.04461508  0.08460748  0.02434039 -0.068727...      4
147  [-6.65913662e-03  1.25265658e-01  8.69285986e-...      4

[148 rows x 2 columns]
[[-3.78747880e-02  2.08841249e-01  1.13818914e-01 ... -9.21863019e-02
   7.38245174e-02  4.08274271e-02]
 [-6.49090633e-02  1.95936307e-01  9.58812609e-02 ... -4.86369506e-02
   6.56304806e-02  6.15514219e-02]
 [-3.32880765e-02  2.3

In [8]:
#Splitting data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
print(X_train.shape)
print(y_train.shape)

(118, 128)
(118,)


In [9]:
#Feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)

X_test = sc.transform(X_test)
print(X_train.shape)
print(y_train.shape)

(118, 128)
(118,)


In [12]:
#Creating a badic ANN model
import tensorflow as tf
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units = 128, activation = 'relu', input_shape = (128,)))
ann.add(tf.keras.layers.Dropout(0.2))
ann.add(tf.keras.layers.Dense(units=5, activation='softmax'))
ann.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 645       
Total params: 17,157
Trainable params: 17,157
Non-trainable params: 0
_________________________________________________________________


In [13]:
#Training the model
ann.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, y_train, batch_size = 32, epochs = 15)

Epoch 1/15
4/4 [==============================] - 0s 1ms/step - loss: 1.6448 - accuracy: 0.3475
Epoch 2/15
4/4 [==============================] - 0s 1ms/step - loss: 0.8369 - accuracy: 0.7203
Epoch 3/15
4/4 [==============================] - 0s 1ms/step - loss: 0.4390 - accuracy: 0.9153
Epoch 4/15
4/4 [==============================] - 0s 2ms/step - loss: 0.1977 - accuracy: 0.9746
Epoch 5/15
4/4 [==============================] - 0s 1ms/step - loss: 0.1400 - accuracy: 0.9831
Epoch 6/15
4/4 [==============================] - 0s 1ms/step - loss: 0.0748 - accuracy: 0.9915
Epoch 7/15
4/4 [==============================] - 0s 2ms/step - loss: 0.0478 - accuracy: 1.0000
Epoch 8/15
4/4 [==============================] - 0s 1ms/step - loss: 0.0360 - accuracy: 1.0000
Epoch 9/15
4/4 [==============================] - 0s 2ms/step - loss: 0.0249 - accuracy: 1.0000
Epoch 10/15
4/4 [==============================] - 0s 1ms/step - loss: 0.0215 - accuracy: 1.0000
Epoch 11/15
4/4 [======================

In [15]:
#Computing predictions on test dataset
y_pred = ann.predict(X_test)
labels = (y_pred > 0.5).astype(np.int)

#Calculating accuracy
predictions=np.zeros(shape=())
for i in labels:
    predictions=np.append(predictions,np.argmax(i))
predictions=np.delete(predictions,0)

accuracy = np.sum(y_test==predictions)/len(y_test)
print(accuracy)

[[1 0 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 0 1 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 0 0 0 1]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [1 0 0 0 0]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 0 1 0]
 [0 0 1 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 0 0 1 0]
 [0 0 1 0 0]
 [1 0 0 0 0]
 [0 0 0 0 1]
 [0 0 0 1 0]
 [0 0 0 1 0]
 [0 0 1 0 0]
 [0 1 0 0 0]
 [0 0 0 1 0]
 [0 0 0 1 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]
Arnold_Schwarzenegger
Arnold_Schwarzenegger
Bill_Clinton
Ricardo_lagos
Arnold_Schwarzenegger
Arnold_Schwarzenegger
Bill_Clinton
Bill_Clinton
Vladimir_Putin
Carlos_menem
Carlos_menem
Arnold_Schwarzenegger
Carlos_menem
Vladimir_Putin
Ricardo_lagos
Carlos_menem
Bill_Clinton
Arnold_Schwarzenegger
Ricardo_lagos
Carlos_menem
Arnold_Schwarzenegger
Vladimir_Putin
Ricardo_lagos
Ricardo_lagos
Carlos_menem
Bill_Clinton
Ricardo_lagos
Ricardo_lagos
Ricardo_lagos
Vladimir_Putin


In [42]:
#Helper functin to run the encoding through the network to and convert the output into corresponding name
def name_helper(enc):
    enc=enc.reshape(1,128)
    new_pred = ann.predict(enc)
    #print(new_pred)
    l = (new_pred > 0.24).astype(np.int)
    l=l.flatten() 
    #print(l)
    #print(np.argmax(l))
    if(np.amax(l)!=0):
        print('yay')
        name=(label[np.argmax(l)])
    else:
        print('no')
        name="random"
    return name


In [36]:
label=['Arnold_Schwarzenegger','Bill_Clinton','Carlos_menem','Ricardo_lagos','Vladimir_Putin']

#Testing on a single image
path='C:/c_program/face/download.jpg'
enc=get_encoding(path)
enc=enc.reshape(1,128)
#print(enc.shape)
name=name_helper(enc)
print(name)

(1, 128)
[[0.1689814  0.24921131 0.1845729  0.23146223 0.16577211]]
[0 1 0 0 0]
1
yay
Bill_Clinton


In [43]:
#Deploying model on the webcam
cap=cv2.VideoCapture(0)
while True:
    s,img=cap.read()
    i=cv2.resize(img,(0,0),None,0.25,0.25)
    i=cv2.cvtColor(i,cv2.COLOR_BGR2RGB)
    faces=face_recognition.face_locations(i)
    encoding=face_recognition.face_encodings(i,faces)
    
    k= cv2.waitKey(1)
    if k == ord('q'):
        break
    
    for encodeface,loc in zip(encoding,faces):
        name=name_helper(encodeface)
        print(name)
        y1,x2,y2,x1=loc
        y1,x2,y2,x1=y1*4,x2*4,y2*4,x1*4
        cv2.rectangle(img,(x2,y2),(x1,y1),(255,0,0),2)
        cv2.rectangle(img,(x1,y2-35),(x2,y2),(255,0,0),cv2.FILLED)
        cv2.putText(img,str(name),(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
            
        cv2.imshow('webcam',img)
        cv2.waitKey(1)
    
cap.release()
cv2.destroyAllWindows()


no
random
no
random
no
random
no
random
no
random
no
random
